In [102]:
# Importing Packages
import pandas as pd
import numpy as np
import os

# NLP Packages
import nltk 
from nltk.corpus import stopwords
from textblob import TextBlob 
from textblob import Word
import re
import string

# WordCloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Sklearn Packages
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning

# ImbLearn Packages
from imblearn.over_sampling import SMOTE

# Pandas Settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Solve warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Import pickle


In [103]:
df = pd.read_csv('../csv/Hotel_Review_Spell_Checked.csv', index_col=0)

# Preprocessing

In [104]:
stop_words = stopwords.words('english')

In [105]:
df.isna().sum()

Unnamed: 0.1       0
Hotel_Name         0
Negative_Review    0
Positive_Review    0
Reviewer_Score     0
Reviews_Clean      0
Score              0
Spell_Checked      0
dtype: int64

In [106]:
df.dropna(inplace=True)

## Count Vectorizer

In [107]:
# Instantiate CountVectorizer
cv = CountVectorizer(stop_words=stop_words)

# Fit and transform dataframe without data cleaning
df_cv = cv.fit_transform(df['Spell_Checked'])
df_cv = pd.DataFrame(df_cv.toarray(), columns = cv.get_feature_names())
df_cv.index = df.index

In [108]:
# Using Sparse in the DataFrame
df_sparse = df_cv.astype('Sparse')

In [109]:
y = df.Score
X_cv = df_cv

In [110]:
# Running Train Test Split
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_cv, y, test_size= 0.25)

## TF-IDF

In [111]:
def fn_tdm_tfidf(docs, xColNames = None, **kwargs):

    #initialize the  vectorizer
    tf = TfidfVectorizer(**kwargs)
    x1 = tf.fit_transform(docs)
    #create dataFrame
    df = pd.DataFrame(x1.toarray().transpose(), index = tf.get_feature_names())

    if xColNames is not None:
        df.columns = xColNames

    return df

In [112]:
df_tfidf = fn_tdm_tfidf(df['Spell_Checked']).transpose()

In [113]:
df_tfidf.drop(columns='zafirovski', inplace=True)

In [114]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(df_tfidf, y, test_size=0.25, random_state=1)

## Evaluation Metric

In [145]:
# Evaluation function

def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Precision: ' + str(metrics.precision_score(y_true, y_pred)))
    print('Accuracy: ' + str(metrics.accuracy_score(y_true, y_pred)))
    print('F1 Score Weighted: ' + str(metrics.f1_score(y_true, y_pred, average="weighted")))
    print('F1 Score Macro: ' + str(metrics.f1_score(y_true, y_pred, average="macro")))
    print('Recall: ' + str(metrics.recall_score(y_true, y_pred)))

## Modeling With Count Vectorizer

### Logistic Regression

In [116]:
# Baseline Regression Model
lg_base = LogisticRegression()
lg_base.fit(X_train_cv, y_train_cv) 
y_lg_base_cv = lg_base.predict(X_test_cv)

# 28 seconds

In [146]:
# Logistic Regression baseline evaluation
evaluation(y_test_cv, y_lg_base_cv)

Evaluation Metrics:
Precision: 0.8070921985815603
Accuracy: 0.8072155411655875
F1 Score Weighted: 0.8071580988932364
F1 Score Macro: 0.8069795283509281
Recall: 0.8204758471521268


### Random Forest

In [118]:
rf_cv = RandomForestClassifier()
rf_cv.fit(X_train_cv, y_train_cv)
y_rf_cv = rf_cv.predict(X_test_cv)

# 50 seconds

In [147]:
# Random Forest baseline evaluation
evaluation(y_test_cv, y_rf_cv)

Evaluation Metrics:
Precision: 0.8269157453297751
Accuracy: 0.8040703052728955
F1 Score Weighted: 0.8040622035618255
F1 Score Macro: 0.8040698760431699
Recall: 0.7819033886085076


### Naive Bayes

In [120]:
nb_base_cv = GaussianNB()
nb_base_cv.fit(X_train_cv, y_train_cv)

y_nb_base_cv = nb_base_cv.predict(X_test_cv)

In [148]:
# Naive Bayes baseline evaluation
evaluation(y_test_cv, y_nb_base_cv)

Evaluation Metrics:
Precision: 0.5555555555555556
Accuracy: 0.5774283071230343
F1 Score Weighted: 0.5302148800880988
F1 Score Macro: 0.5260880340819889
Recall: 0.8832011535688536


## Modeling with TF-IDF

### Logistic Regression

In [124]:
# Baseline Regression Model
lg_base_tfidf = LogisticRegression()
lg_base_tfidf.fit(X_train_tfidf, y_train_tfidf) 
y_lg_base_tfidf = lg_base_tfidf.predict(X_test_tfidf)
# 28 seconds

In [149]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test_tfidf, y_lg_base_tfidf)

Evaluation Metrics:
Precision: 0.8516104294478528
Accuracy: 0.8210915818686402
F1 Score Weighted: 0.8210963096449203
F1 Score Macro: 0.8210914838836396
Recall: 0.7929310960371296


### Random Forest 

In [126]:
rf_base_tfidf = RandomForestClassifier()
rf_base_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_rf_base_tfidf = rf_base_tfidf.predict(X_test_tfidf)

# 50 seconds

In [150]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test_tfidf, y_rf_base_tfidf)

Evaluation Metrics:
Precision: 0.8422098569157392
Accuracy: 0.8003700277520814
F1 Score Weighted: 0.8001986016227505
F1 Score Macro: 0.8003170961753056
Recall: 0.7565155301677973


## TF-IDF With Lemmatization

### Create train and test set

In [138]:
X_lem = pickle.load(open('../pickle/X_lem.pkl', 'rb'))
y_lem = pd.read_pickle('../pickle/y_lem.pkl')

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X_lem, y_lem, test_size=0.20, random_state=15)

In [140]:
# using tf_idf vectorizor
tfidf = TfidfVectorizer(stop_words= stop_words, ngram_range=(1,2))

# sparse matrix format with 265K stored elements
tfidf_data_train = tfidf.fit_transform(X_train)
tfidf_data_test = tfidf.transform(X_test)

### Logistic Regression

In [143]:
# Baseline Regression Model
lg_base_tfidf = LogisticRegression()
lg_base_tfidf.fit(tfidf_data_train, y_train) 
y_lg_base_tfidf_lem = lg_base_tfidf.predict(tfidf_data_test)
# 28 seconds

In [151]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test, y_lg_base_tfidf_lem)

Evaluation Metrics:
Precision: 0.8558648111332008
Accuracy: 0.8270120259019427
F1 Score Weighted: 0.8268066579512767
F1 Score Macro: 0.8268506653692765
Recall: 0.789908256880734


### Random Forest

In [152]:
rf_base_tfidf = RandomForestClassifier()
rf_base_tfidf.fit(tfidf_data_train, y_train)
y_rf_base_tfidf = rf_base_tfidf.predict(tfidf_data_test)

In [153]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test, y_rf_base_tfidf)

Evaluation Metrics:
Precision: 0.828941063727839
Accuracy: 0.8133672525439408
F1 Score Weighted: 0.8133143288977217
F1 Score Macro: 0.8133348154704517
Recall: 0.7935779816513762


### GridSearch LogReg

In [135]:
#This is the parameters for the baseline Gridsearch with Logistic Regression
param_dict={
    "class_weight":["balanced"],
    "solver":["lbfgs","liblinear"]
}
logreg = LogisticRegression(max_iter=1000)

In [ ]:
grid_lg = GridSearchCV(logreg, param_dict, cv=20, scoring='f1_weighted', verbose=1, n_jobs=-1)
grid_lg.fit(X_train_cv,y_train_cv)

In [ ]:
y_gc_base_cv = grid_lg(X_test)

In [ ]:
# Logistic Regression baseline evaluation
evaluation(y_test_cv, y_gc_base_cv)